# Практическая работа Урок 4. Система управления базами данных MongoDB в Python

Задание 1.Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru. Для парсинга использовать XPath. 

Структура данных должна содержать:
- название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

Задание 2. Сложить собранные новости в БД

In [1]:
import sys
import requests
from lxml import html
from pprint import pprint
from datetime import datetime, date
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['lenta_ru_news']
news = db.news

In [3]:
def add_to_db(doc, only_new = True):
    exists = news.find_one({'link': doc['link']})
    if only_new == True:
        if not exists:
            try:
                news.insert_one(doc)
            except DuplicateKeyError:
                pass
    else:
        try:
            news.insert_one(doc)
        except DuplicateKeyError:
            pass

In [4]:
url = 'https://lenta.ru/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}
session = requests.Session()
response = session.get(url, headers=headers)

In [5]:
dom = html.fromstring(response.text)

items = dom.xpath("//a[contains(@class,'card-mini')]")

for item in items:
    new = {}
    source = url
    name = item.xpath(".//h3[@class='card-mini__title']/text()")
    link = url + str(item.xpath("./@href"))
    time = item.xpath(".//time[@class='card-mini__info-item']/text()")

    new['source'] = source
    new['name'] = name
    new['link'] = link
    if len(''.join(time))<15:
        new['time'] = ''.join(time) + ', ' + date.today().strftime("%d %b %y")
    else:
        new['time'] = ''.join(new)



    add_to_db(new, only_new = True)

In [6]:
for item in news.find({'source': url}):
    pprint(item)

{'_id': ObjectId('64cbb89482bf3267c3141bab'),
 'link': "https://lenta.ru/['/news/2023/08/03/amoralno/']",
 'name': ['Депутат Рады обвинил Зеленского в аморальности в ответ на обыски'],
 'source': 'https://lenta.ru/',
 'time': '17:22, 03 Aug 23'}
{'_id': ObjectId('64cbb89482bf3267c3141bac'),
 'link': "https://lenta.ru/['/news/2023/08/03/aer/']",
 'name': ['Россиянам дали советы на случай потери багажа в аэропорту'],
 'source': 'https://lenta.ru/',
 'time': '17:20, 03 Aug 23'}
{'_id': ObjectId('64cbb89482bf3267c3141bad'),
 'link': "https://lenta.ru/['/news/2023/08/03/novak/']",
 'name': ['Россия пообещала продавать меньше нефти за рубеж'],
 'source': 'https://lenta.ru/',
 'time': '17:14, 03 Aug 23'}
{'_id': ObjectId('64cbb89482bf3267c3141bae'),
 'link': "https://lenta.ru/['/news/2023/08/03/drake/']",
 'name': ['Рэпер Дрейк стал рекордсменом по заработку за один концерт'],
 'source': 'https://lenta.ru/',
 'time': '17:13, 03 Aug 23'}
{'_id': ObjectId('64cbb89482bf3267c3141baf'),
 'link': "